# Torch Classification 

Imvolves the following steps

1. Import necessary libraries
1. Generate synthetic data for Classifiation case and split the dataset
1. Load data into loader into batches
1. Create a simple model
1. Initialize optimizer and loss function
1. Strat  Training 
1. Draw the plots and Decision surface

# Import necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_moons, make_regression
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset

In [4]:
# Generate synthetic classification dataset (moons)
n_samples = 1000
X_cls, y_cls = make_moons(n_samples=n_samples, noise=0.2, random_state=42)
X_cls = torch.tensor(X_cls, dtype=torch.float32)
y_cls = torch.tensor(y_cls, dtype=torch.long)


# Split the dataset

In [5]:
# Split the dataset into train, val, test sets (70%, 20%, 10%)
X_train_cls, X_temp_cls, y_train_cls, y_temp_cls = train_test_split(X_cls, y_cls, test_size=0.3, random_state=42)
X_val_cls, X_test_cls, y_val_cls, y_test_cls = train_test_split(X_temp_cls, y_temp_cls, test_size=0.33, random_state=42)


# Load the dataset into batches

In [ ]:
# Create DataLoader for batch processing
train_dataset_cls = TensorDataset(X_train_cls, y_train_cls)
val_dataset_cls = TensorDataset(X_val_cls, y_val_cls)
test_dataset_cls = TensorDataset(X_test_cls, y_test_cls)

train_loader_cls = DataLoader(train_dataset_cls, batch_size=32, shuffle=True)
val_loader_cls = DataLoader(val_dataset_cls, batch_size=32, shuffle=False)
test_loader_cls = DataLoader(test_dataset_cls, batch_size=32, shuffle=False)


# Get a simple model

In [ ]:
# Define a simple neural network model for classification
class SimpleClassificationModel(nn.Module):
    def __init__(self):
        super(SimpleClassificationModel, self).__init__()
        self.linear1 = nn.Linear(2, 1)  # 2 input features, 1 output (binary classification)

    def forward(self, x):
        return torch.sigmoid(self.linear1(x))

# Initialize optimizer and loss function

In [ ]:
# Initialize the classification model, loss function, and optimizer
model_cls = SimpleClassificationModel()
criterion_cls = nn.BCELoss()
optimizer_cls = optim.Adam(model_cls.parameters(), lr=0.01)

# Start training

In [ ]:
epochs_cls = 500
losses_train_cls = []
losses_val_cls = []
accuracies_train_cls = []
accuracies_val_cls = []

for epoch in range(epochs_cls):
    model_cls.train()
    epoch_losses_train = []
    epoch_correct_train = 0
    epoch_total_train = 0

    # Training phase
    for inputs, targets in train_loader_cls:
        optimizer_cls.zero_grad()
        outputs_train = model_cls(inputs)
        loss_train = criterion_cls(outputs_train, targets.float().view(-1, 1))
        loss_train.backward()
        optimizer_cls.step()
        epoch_losses_train.append(loss_train.item())

        # Calculate training accuracy
        predicted_train = (outputs_train > 0.5).float()
        epoch_correct_train += (predicted_train == targets.float().view_as(predicted_train)).sum().item()
        epoch_total_train += targets.size(0)

    # Calculate mean loss and accuracy for the epoch
    losses_train_cls.append(np.mean(epoch_losses_train))
    accuracy_train = epoch_correct_train / epoch_total_train
    accuracies_train_cls.append(accuracy_train)

    # Evaluation phase (validation)
    model_cls.eval()
    epoch_losses_val = []
    epoch_correct_val = 0
    epoch_total_val = 0

    with torch.no_grad():
        for inputs, targets in val_loader_cls:
            outputs_val = model_cls(inputs)
            loss_val = criterion_cls(outputs_val, targets.float().view(-1, 1))
            epoch_losses_val.append(loss_val.item())

            # Calculate validation accuracy
            predicted_val = (outputs_val > 0.5).float()
            epoch_correct_val += (predicted_val == targets.float().view_as(predicted_val)).sum().item()
            epoch_total_val += targets.size(0)

    # Calculate mean loss and accuracy for validation set
    losses_val_cls.append(np.mean(epoch_losses_val))
    accuracy_val = epoch_correct_val / epoch_total_val
    accuracies_val_cls.append(accuracy_val)

    # Print progress every 50 epochs
    if epoch % 50 == 0:
        print(f'Epoch [{epoch}/{epochs_cls}], Train Loss: {losses_train_cls[-1]:.4f}, Val Loss: {losses_val_cls[-1]:.4f}, '
              f'Train Acc: {accuracy_train:.4f}, Val Acc: {accuracy_val:.4f}')


# Plot loss, acc and decision surface

In [ ]:
# Plot losses over epochs
plt.figure(figsize=(12, 5))

# Plot losses
plt.subplot(1, 2, 1)
plt.plot(losses_train_cls, label='Training Loss')
plt.plot(losses_val_cls, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Classification Training and Validation Losses')
plt.legend()
plt.grid(True)

# Plot accuracies
plt.subplot(1, 2, 2)
plt.plot(accuracies_train_cls, label='Training Accuracy')
plt.plot(accuracies_val_cls, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Classification Training and Validation Accuracies')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Generate a meshgrid for decision contour plot
x_min, x_max = X_cls[:, 0].min() - 0.5, X_cls[:, 0].max() + 0.5
y_min, y_max = X_cls[:, 1].min() - 0.5, X_cls[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

# Create input tensor from meshgrid for prediction
mesh_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
model_cls.eval()
with torch.no_grad():
    outputs = model_cls(mesh_tensor)
    outputs_pred = (outputs >= 0.5).float()  # Convert to class labels 0 or 1

# Reshape outputs_pred to match xx and yy dimensions for plotting
Z = outputs_pred.numpy().reshape(xx.shape)

# Plot decision contours with different colors for each class and add colorbar
plt.figure(figsize=(8, 6))
contour = plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)

# Plot data points
plt.scatter(X_cls[:, 0], X_cls[:, 1], c=y_cls, cmap=plt.cm.Paired, edgecolors='k')
plt.title('Decision Boundary for Classification')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True)
plt.show()